In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge
warnings.simplefilter(action='ignore', category=FutureWarning)
import gc

In [2]:
#this is used to reduce the memory usage of the dataframe 
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
#Read the csv 
new_transactions = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv', parse_dates=['purchase_date'])
historical_transactions = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv', parse_dates=['purchase_date'])

def binarize(df):
    """
    Parameters
    -----------
    df : input dataframe 
    
    Return 
    ----------
    dataframe with column authorized_flag and category_1 ,mapped to either 1 or 0 """
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [4]:
%%time
def read_data(input_file):
    #change the column type to date type
    #calculate the elapsed time 
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df

train = read_data('../input/elo-merchant-category-recommendation/train.csv')
test = read_data('../input/elo-merchant-category-recommendation/test.csv')

#take out the target column in the training data 
target = train['target']
del train['target']
gc.collect()

CPU times: user 1.6 s, sys: 4 ms, total: 1.6 s
Wall time: 1.61 s


In [5]:
historical_transactions['purchase_amount_new'] = np.round(historical_transactions['purchase_amount'] / 0.00150265118 + 497.06,2)
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,29.00
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,9.17
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,17.65
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,7.69
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,16.00


In [6]:
train.head()



,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92


In [7]:
historical_transactions.sort_values(['card_id','purchase_date'],inplace = True,ascending= False)

In [8]:
gc.collect()
historical_transactions.head(50)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new
22821209,1,C_ID_fffffd5772,-1,1,1,B,661,M_ID_fc7d7969c3,0,-0.745405,2018-02-26 06:58:17,NaN,-1,8,1.00
22821184,0,C_ID_fffffd5772,-1,1,1,B,661,M_ID_fc7d7969c3,0,-0.745405,2018-02-26 06:57:39,NaN,-1,8,1.00
22821171,0,C_ID_fffffd5772,-1,1,1,B,661,M_ID_fc7d7969c3,0,-0.745405,2018-02-26 06:56:49,NaN,-1,8,1.00
22821210,0,C_ID_fffffd5772,-1,1,1,B,661,M_ID_fc7d7969c3,0,-0.745405,2018-02-26 06:56:26,NaN,-1,8,1.00
22821237,0,C_ID_fffffd5772,-1,1,1,B,661,M_ID_fc7d7969c3,0,-0.745405,2018-02-26 06:55:45,NaN,-1,8,1.00
22821189,1,C_ID_fffffd5772,20,0,1,B,307,M_ID_55cb8d33dc,0,-0.671775,2018-02-25 09:38:37,3.0,19,19,50.00
22821170,1,C_ID_fffffd5772,20,0,1,B,705,M_ID_d6626c9a19,0,-0.689281,2018-02-23 14:13:52,3.0,19,33,38.35
22821160,1,C_ID_fffffd5772,20,0,1,B,560,M_ID_3841612e17,0,-0.676524,2018-02-23 14:12:15,3.0,19,34,46.84
22821181,1,C_ID_fffffd5772,20,0,1,B,307,M_ID_55cb8d33dc,0,-0.686802,2018-02-23 14:02:46,3.0,19,19,40.00
22821166,1,C_ID_fffffd5772,-1,1,1,B,755,M_ID_445742726b,0,-0.728876,2018-02-23 08:54:58,NaN,-1,8,12.00


In [115]:
#two level group by ['card_id ' , 'month_lag']
running_sum = historical_transactions.groupby(['card_id','month_lag'],as_index=False)['purchase_amount_new'].sum()
# historical_transactions['month_lag'].min()
running_sum.head()

,card_id,month_lag,purchase_amount_new
0,C_ID_00007093c1,-12,1100.20
1,C_ID_00007093c1,-11,1168.90
2,C_ID_00007093c1,-10,1633.25
3,C_ID_00007093c1,-9,2691.13
4,C_ID_00007093c1,-8,2876.63


In [116]:
running_sum.head()

,card_id,month_lag,purchase_amount_new
0,C_ID_00007093c1,-12,1100.20
1,C_ID_00007093c1,-11,1168.90
2,C_ID_00007093c1,-10,1633.25
3,C_ID_00007093c1,-9,2691.13
4,C_ID_00007093c1,-8,2876.63


In [117]:
wide_running_sum = running_sum.pivot(index= 'card_id',columns= 'month_lag',values = 'purchase_amount_new')

In [118]:
wide_running_sum.head()

month_lag,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
card_id,,,,,,,,,,,,,,
C_ID_00007093c1,NaN,1100.20,1168.90,1633.25,2691.13,2876.63,1307.79,4189.00,524.57,1513.59,940.25,1985.22,1186.59,1805.18
C_ID_0001238066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.13,943.03,1831.55,4121.19,3884.94,2017.62
C_ID_0001506ef0,51.0,184.48,286.53,NaN,30.00,24.87,1017.00,849.50,770.50,910.74,1089.28,1579.29,1365.30,1620.25
C_ID_0001793786,NaN,NaN,NaN,NaN,1269.83,3745.99,7389.91,7783.04,3380.25,14441.70,13971.57,9683.56,14936.02,6282.35
C_ID_000183fdda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2831.40,1675.01,2326.11,2237.65,4312.91,8872.47,3510.10


In [119]:
wide_running_sum.reset_index(drop = False,inplace = True)
wide_running_sum.head()

month_lag,card_id,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
0,C_ID_00007093c1,NaN,1100.20,1168.90,1633.25,2691.13,2876.63,1307.79,4189.00,524.57,1513.59,940.25,1985.22,1186.59,1805.18
1,C_ID_0001238066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.13,943.03,1831.55,4121.19,3884.94,2017.62
2,C_ID_0001506ef0,51.0,184.48,286.53,NaN,30.00,24.87,1017.00,849.50,770.50,910.74,1089.28,1579.29,1365.30,1620.25
3,C_ID_0001793786,NaN,NaN,NaN,NaN,1269.83,3745.99,7389.91,7783.04,3380.25,14441.70,13971.57,9683.56,14936.02,6282.35
4,C_ID_000183fdda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2831.40,1675.01,2326.11,2237.65,4312.91,8872.47,3510.10


In [120]:
wide_running_sum['last_six_month'] = wide_running_sum.iloc[:,-7:].sum(axis=1)
wide_running_sum['first_six_month'] = wide_running_sum.iloc[:,2:9].sum(axis=1)
wide_running_sum['ratio'] =wide_running_sum['first_six_month']/wide_running_sum['last_six_month']
# df['Fruit Total']=df.iloc[:,-4:].sum(axis=1)
wide_running_sum.head()

month_lag,card_id,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,last_six_month,first_six_month,ratio
0,C_ID_00007093c1,NaN,1100.20,1168.90,1633.25,2691.13,2876.63,1307.79,4189.00,524.57,1513.59,940.25,1985.22,1186.59,1805.18,12144.40,14966.90,1.232412
1,C_ID_0001238066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.13,943.03,1831.55,4121.19,3884.94,2017.62,12925.46,0.00,0.000000
2,C_ID_0001506ef0,51.0,184.48,286.53,NaN,30.00,24.87,1017.00,849.50,770.50,910.74,1089.28,1579.29,1365.30,1620.25,8184.86,2392.38,0.292293
3,C_ID_0001793786,NaN,NaN,NaN,NaN,1269.83,3745.99,7389.91,7783.04,3380.25,14441.70,13971.57,9683.56,14936.02,6282.35,70478.49,20188.77,0.286453
4,C_ID_000183fdda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2831.40,1675.01,2326.11,2237.65,4312.91,8872.47,3510.10,25765.65,2831.40,0.109890


In [121]:


wide_running_sum.iloc[:,1:3].head()

month_lag,-13,-12
0,NaN,1100.20
1,NaN,NaN
2,51.0,184.48
3,NaN,NaN
4,NaN,NaN


In [122]:
i = 2
wide_running_sum.iloc[:,3:5].head()

month_lag,-11,-10
0,1168.90,1633.25
1,NaN,NaN
2,286.53,NaN
3,NaN,NaN
4,NaN,NaN


In [123]:
#create a for loop for all these shit 


step = 1


while step <7 : 
    print ('value of step : {}'.format(step))
    inter_col1 = 'first' + str(step)
    inter_col2 = 'last' + str(step)
    final_col = 'ratio' +str(step)
    wide_running_sum[inter_col1] = wide_running_sum.iloc[:,1:(2+step)].sum(axis=1)
    print('wide_running_sum[inter_col1] = wide_running_sum.iloc[:,1:{}].sum(axis=1)'.format(2+step))
    wide_running_sum[inter_col2] = wide_running_sum.iloc[:,(2+step):(2+2*step)].sum(axis=1)
    print('wide_running_sum[inter_col2] = wide_running_sum.iloc[:,{}:{}].sum(axis=1)'.format(2+step,2+2*step))
    wide_running_sum[final_col] = wide_running_sum[inter_col2] / wide_running_sum[inter_col1]
    
    del wide_running_sum[inter_col1]
    del wide_running_sum[inter_col2]
    step +=1

wide_running_sum.head(20)

value of step : 1
wide_running_sum[inter_col1] = wide_running_sum.iloc[:,1:3].sum(axis=1)
wide_running_sum[inter_col2] = wide_running_sum.iloc[:,3:4].sum(axis=1)
value of step : 2
wide_running_sum[inter_col1] = wide_running_sum.iloc[:,1:4].sum(axis=1)
wide_running_sum[inter_col2] = wide_running_sum.iloc[:,4:6].sum(axis=1)
value of step : 3
wide_running_sum[inter_col1] = wide_running_sum.iloc[:,1:5].sum(axis=1)
wide_running_sum[inter_col2] = wide_running_sum.iloc[:,5:8].sum(axis=1)
value of step : 4
wide_running_sum[inter_col1] = wide_running_sum.iloc[:,1:6].sum(axis=1)
wide_running_sum[inter_col2] = wide_running_sum.iloc[:,6:10].sum(axis=1)
value of step : 5
wide_running_sum[inter_col1] = wide_running_sum.iloc[:,1:7].sum(axis=1)
wide_running_sum[inter_col2] = wide_running_sum.iloc[:,7:12].sum(axis=1)
value of step : 6
wide_running_sum[inter_col1] = wide_running_sum.iloc[:,1:8].sum(axis=1)
wide_running_sum[inter_col2] = wide_running_sum.iloc[:,8:14].sum(axis=1)


month_lag,card_id,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,last_six_month,first_six_month,ratio,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6
0,C_ID_00007093c1,NaN,1100.20,1168.90,1633.25,2691.13,2876.63,1307.79,4189.00,524.57,1513.59,940.25,1985.22,1186.59,1805.18,12144.40,14966.90,1.232412,1.062443,1.905769,1.761900,1.349513,0.894942,0.959298
1,C_ID_0001238066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.13,943.03,1831.55,4121.19,3884.94,2017.62,12925.46,0.00,0.000000,NaN,NaN,NaN,inf,inf,inf
2,C_ID_0001506ef0,51.00,184.48,286.53,NaN,30.00,24.87,1017.00,849.50,770.50,910.74,1089.28,1579.29,1365.30,1620.25,8184.86,2392.38,0.292293,1.216791,0.057470,2.053351,4.822141,8.038102,4.118635
3,C_ID_0001793786,NaN,NaN,NaN,NaN,1269.83,3745.99,7389.91,7783.04,3380.25,14441.70,13971.57,9683.56,14936.02,6282.35,70478.49,20188.77,0.286453,NaN,inf,inf,17.560768,9.363667,5.174717
4,C_ID_000183fdda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2831.40,1675.01,2326.11,2237.65,4312.91,8872.47,3510.10,25765.65,2831.40,0.109890,NaN,NaN,NaN,inf,inf,inf
5,C_ID_00024e244b,111.84,8.48,29.99,189.48,223.84,NaN,100.83,5593.30,280.58,40.69,18.00,354.83,100.52,88.86,6476.78,6145.92,0.948916,0.249252,2.749784,0.955502,10.600412,10.704540,9.613701
6,C_ID_0002709b5a,36.37,NaN,NaN,53.15,446.14,400.34,361.02,473.33,188.98,461.95,420.58,549.70,823.85,353.00,3271.39,1733.98,0.530044,0.000000,13.728073,13.488606,2.657787,2.036175,2.250073
7,C_ID_00027503e2,NaN,NaN,NaN,NaN,NaN,4.00,18.18,31.62,20.87,13.95,5.73,32.00,19.74,18.34,142.25,53.80,0.378207,NaN,NaN,inf,inf,22.587500,5.586564
8,C_ID_000298032a,NaN,NaN,37.72,163.95,211.32,NaN,NaN,206.93,NaN,130.00,144.99,338.99,398.84,2636.26,3856.01,619.92,0.160767,inf,9.948834,1.047850,0.501053,1.166905,2.953461
9,C_ID_0002ba3c2e,NaN,NaN,NaN,NaN,256.41,NaN,458.99,270.90,NaN,NaN,1536.24,1242.13,1040.38,479.31,4568.96,986.30,0.215870,NaN,inf,inf,2.846574,8.837916,5.716592


In [124]:
wide_running_sum.replace([np.inf, -np.inf], np.nan,inplace = True)
wide_running_sum.fillna(-33.219281,inplace = True)
wide_running_sum.head()

month_lag,card_id,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,last_six_month,first_six_month,ratio,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6
0,C_ID_00007093c1,-33.219281,1100.200000,1168.900000,1633.250000,2691.130000,2876.630000,1307.790000,4189.000000,524.57,1513.59,940.25,1985.22,1186.59,1805.18,12144.40,14966.90,1.232412,1.062443,1.905769,1.761900,1.349513,0.894942,0.959298
1,C_ID_0001238066,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,127.13,943.03,1831.55,4121.19,3884.94,2017.62,12925.46,0.00,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
2,C_ID_0001506ef0,51.000000,184.480000,286.530000,-33.219281,30.000000,24.870000,1017.000000,849.500000,770.50,910.74,1089.28,1579.29,1365.30,1620.25,8184.86,2392.38,0.292293,1.216791,0.057470,2.053351,4.822141,8.038102,4.118635
3,C_ID_0001793786,-33.219281,-33.219281,-33.219281,-33.219281,1269.830000,3745.990000,7389.910000,7783.040000,3380.25,14441.70,13971.57,9683.56,14936.02,6282.35,70478.49,20188.77,0.286453,-33.219281,-33.219281,-33.219281,17.560768,9.363667,5.174717
4,C_ID_000183fdda,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,2831.400000,1675.01,2326.11,2237.65,4312.91,8872.47,3510.10,25765.65,2831.40,0.109890,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281


In [125]:
train.head()


,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92


In [126]:
merge_df = train.merge(right =wide_running_sum,how = 'left', on ='card_id')

merge_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,last_six_month,first_six_month,ratio,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6
0,2017-06-01,C_ID_92a2005557,5,2,1,245,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,112.750000,2510.000000,1252.170000,1432.910000,1344.690000,1444.360000,8178.19,1425.84,1084.08,16162.24,3874.92,0.239751,-33.219281,-33.219281,-33.219281,-33.219281,70.812683,5.748989
1,2017-01-01,C_ID_3d0044924f,4,1,0,396,-33.219281,5511.950000,2633.930000,2729.940000,2050.800000,1429.770000,1897.700000,3300.850000,1674.260000,5669.880000,2137.240000,901.55,2256.74,2019.18,17959.70,19554.94,1.088823,0.477858,0.586891,0.494516,0.642285,1.022536,0.980708
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549,206.420000,253.810000,116.220000,273.730000,158.740000,-33.219281,66.380000,71.340000,29.860000,263.860000,-33.219281,26.02,439.60,56.98,887.66,940.22,1.059212,0.252526,0.750230,0.264791,0.166098,0.427626,0.772510
3,2017-09-01,C_ID_186d6a6901,4,3,0,153,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,243.720000,1254.030000,2228.440000,362.29,453.84,795.27,5337.59,0.00,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,3866.920000,2161.79,3656.80,24022.30,33707.81,0.00,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281


In [127]:
merge_df['target'] = target
merge_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,last_six_month,first_six_month,ratio,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6,target
0,2017-06-01,C_ID_92a2005557,5,2,1,245,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,112.750000,2510.000000,1252.170000,1432.910000,1344.690000,1444.360000,8178.19,1425.84,1084.08,16162.24,3874.92,0.239751,-33.219281,-33.219281,-33.219281,-33.219281,70.812683,5.748989,-0.820283
1,2017-01-01,C_ID_3d0044924f,4,1,0,396,-33.219281,5511.950000,2633.930000,2729.940000,2050.800000,1429.770000,1897.700000,3300.850000,1674.260000,5669.880000,2137.240000,901.55,2256.74,2019.18,17959.70,19554.94,1.088823,0.477858,0.586891,0.494516,0.642285,1.022536,0.980708,0.392913
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549,206.420000,253.810000,116.220000,273.730000,158.740000,-33.219281,66.380000,71.340000,29.860000,263.860000,-33.219281,26.02,439.60,56.98,887.66,940.22,1.059212,0.252526,0.750230,0.264791,0.166098,0.427626,0.772510,0.688056
3,2017-09-01,C_ID_186d6a6901,4,3,0,153,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,243.720000,1254.030000,2228.440000,362.29,453.84,795.27,5337.59,0.00,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,0.142495
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,3866.920000,2161.79,3656.80,24022.30,33707.81,0.00,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-0.159749


In [128]:
merge_df[merge_df['target'] <-30]

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,last_six_month,first_six_month,ratio,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6,target
64,2017-09-01,C_ID_8186f3fcc1,3,1,1,153,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,124.970000,1931.440000,1858.570000,1869.020000,3267.450000,517.250000,9568.70,0.00,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
92,2015-05-01,C_ID_b9379a30ea,3,3,1,1007,-33.219281,-33.219281,-33.219281,241.230000,65.000000,4.500000,-33.219281,40.810000,29.780000,108.000000,87.050000,-33.219281,-33.219281,-33.219281,265.64,351.54,1.323370,-33.219281,-33.219281,0.288107,0.245208,0.854890,0.854890,-33.219281
125,2015-10-01,C_ID_e9120f535c,3,3,1,854,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,857.360000,744.910000,994.520000,648.730000,1060.870000,1864.760000,2073.190000,8244.34,857.36,0.103994,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
151,2017-03-01,C_ID_65715cb80d,4,2,0,337,-33.219281,-33.219281,-33.219281,32.830000,22.900000,22.900000,175.940000,163.010000,118.480000,283.980000,253.470000,265.310000,273.500000,221.600000,1579.35,417.58,0.264400,-33.219281,-33.219281,6.754188,8.618877,12.652677,5.333504,-33.219281
225,2017-01-01,C_ID_ae77d244b6,2,1,0,396,-33.219281,51.310000,86.850000,92.900000,183.000000,126.900000,194.190000,172.200000,144.950000,177.390000,476.280000,50.000000,-33.219281,30.000000,1050.82,907.35,0.863469,1.692653,1.996960,2.181641,1.541419,2.153597,1.388587,-33.219281
248,2015-07-01,C_ID_c4262c902e,5,1,1,946,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,1188.900000,10586.900000,1540.900000,3132.600000,1208.000000,1655.280000,2084.000000,21396.58,1188.90,0.055565,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
254,2017-03-01,C_ID_6a2c823e5f,2,1,0,337,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,350.800000,400.700000,386.780000,127.800000,289.290000,147.800000,1703.17,0.00,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
386,2017-06-01,C_ID_98e1cb10b6,2,3,0,245,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,1301.480000,2531.940000,7229.840000,1979.310000,388.070000,13430.64,0.00,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
467,2017-03-01,C_ID_eb8175721a,4,1,0,337,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,547.680000,142.830000,1366.040000,1978.470000,2519.630000,1575.940000,168.240000,2089.470000,1405.360000,11103.15,2056.55,0.185222,-33.219281,-33.219281,-33.219281,-33.219281,13.845512,14.044387,-33.219281
515,2016-04-01,C_ID_a2580006bf,4,2,0,671,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,2558.370000,3826.340000,237.720000,559.180000,-33.219281,3337.170000,10518.78,0.00,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281


In [129]:
%%time
def predict_row(row):
    if row['ratio'] ==-33.219281:
        return  -33.219281
    elif row['ratio1'] ==-33.219281:
        return -33.219281
    elif row['ratio2'] ==-33.219281:
        return -33.219281
    elif row['ratio3'] ==-33.219281:
        return  -33.219281
    elif row['ratio4'] ==-33.219281:
        return -33.219281
    elif row['ratio5'] ==-33.219281:
        return -33.219281
    elif row['ratio6'] ==-33.219281:
        return -33.219281
    else:
        return 1
merge_df['predict'] = wide_running_sum.apply (lambda row: predict_row(row),axis=1)
merge_df.head()



CPU times: user 11 s, sys: 148 ms, total: 11.1 s
Wall time: 11.2 s


In [130]:
merge_df['var'] = merge_df['predict'] - merge_df['target']
tung =merge_df[merge_df['target'] <-30]
tung.shape

(2207, 32)

In [131]:
tung['var'].value_counts()

-5.000000e-08    1565
 3.421928e+01     642
Name: var, dtype: int64

In [132]:
test_df = test.merge(right =wide_running_sum,how = 'left', on ='card_id')


In [133]:
test_df.shape

(123623, 29)

In [136]:
test_df.columns

Index(['first_active_month',            'card_id',          'feature_1',
                'feature_2',          'feature_3',       'elapsed_time',
                        -13,                  -12,                  -11,
                        -10,                   -9,                   -8,
                         -7,                   -6,                   -5,
                         -4,                   -3,                   -2,
                         -1,                    0,     'last_six_month',
          'first_six_month',              'ratio',             'ratio1',
                   'ratio2',             'ratio3',             'ratio4',
                   'ratio5',             'ratio6'],
      dtype='object')

In [137]:
cols = ['ratio','ratio1','ratio2','ratio3','ratio4','ratio5','ratio6']
merge_test_df = test_df[cols]
merge_test_df.head()

,ratio,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6
0,0.599084,-33.219281,-33.219281,-33.219281,-33.219281,4.249181,4.176032
1,35.251905,0.884284,1.161797,0.183118,0.000000,0.000000,0.000000
2,0.015805,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
3,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
4,0.233649,0.110617,0.296619,0.224353,1.157250,2.164060,2.911225


In [138]:
merge_test_df['predict'] = wide_running_sum.apply (lambda row: predict_row(row),axis=1)
merge_test_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ratio,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6,predict
0,0.599084,-33.219281,-33.219281,-33.219281,-33.219281,4.249181,4.176032,1.000000
1,35.251905,0.884284,1.161797,0.183118,0.000000,0.000000,0.000000,-33.219281
2,0.015805,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,1.000000
3,0.000000,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281,-33.219281
4,0.233649,0.110617,0.296619,0.224353,1.157250,2.164060,2.911225,-33.219281


In [139]:
merge_test_df[merge_test_df['predict'] ==-33.219281].shape

(89711, 8)

In [140]:
merge_test_df.shape

(123623, 8)

In [ ]:
merge_test_df.to_csv('postprocessing_test.csv',index = False)